# a-6. 欠損値

データは何らかの理由で欠損することがある。もともとのデータに値が入力されなかったり、計算過程でゼロ割り算の結果を欠損とすることもある。
そのような欠損した値を表すときにプログラミングの世界では「null値」という言葉が使われるが、Pandasの中ではfillna()など'NA'(not available)という用語がよく利用されるので、以下ではNAで統一する。
またNAを具体的にどのような値として表現するかは実装系によって異なる。
PythonではNoneという値が使われ、Pandasのfloat型においてはnp.nanという値が使われ、Period型ではNaTが使われる。

---
## 1. 基本
## 2. NAを含む行の削除(dropna)
## 3. NAの置換(fillna, combine_first)
## 4. NAの数値補完(interpolate)
## 5. 条件指定によるNAへの置換

### 関連ドキュメント
- [Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)

---
### APIリファレンス
- [DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- [DataFrame.fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)
- [DataFrame.combine_first](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.combine_first.html)
- [DataFrame.interpolate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html)
- [Series.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dropna.html)
- [Series.fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.fillna.html)
- [Series.combine_first](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.combine_first.html)
- [Series.interpolate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html)
---


In [1]:
import numpy as np
import pandas as pd

---
## 1. 基本

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0 ]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>
 
<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th>備考</th></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.isna())
</pre></td>
<td><img width="60%" src="./figures/missingdata/miss1-1data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss1-1.png"></td>
<td>
isna()メソッドで、値がNAかどうかをTrue/Falseで出力できる。    
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d['x'].isna())
</pre></td>
<td><img width="60%" src="./figures/missingdata/miss1-2data.png"></td>
<td><img width="40%" src="./figures/missingdata/miss1-2.png"></td>
<td>
Seriesに対して利用することも可能。
</td>
</tr>
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.notna())
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss1-3data.png"></td>
<td><img width="80%" src="./figures/missingdata/miss1-3.png"></td>
<td>
notna()メソッドはisna()の逆の動きをする。
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d[d['x'].isna()])
</pre></td>
<td><img width="65%" src="./figures/missingdata/miss1-4data.png"></td>
<td><img width="65%" src="./figures/missingdata/miss1-4.png"></td>
<td>
[]演算子と組み合わせればNAの行を選択することができる。<br>x列がNAの行を選択する例。
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-5</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d * 2)
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss1-5data.png"></td>
<td><img width="65%" src="./figures/missingdata/miss1-5.png"></td>
<td>
NAに対する演算はNA。
</td>
</tr>
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-6</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d > 0)
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss1-6data.png"></td>
<td><img width="75%" src="./figures/missingdata/miss1-6.png"></td>
<td>
NAに対する比較演算はFalse
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-7</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.sum(axis=1))
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss1-7data.png"></td>
<td><img width="65%" src="./figures/missingdata/miss1-7.png"></td>
<td>
sum()などの集約メソッドについてはNAは無視される。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-8</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.sum(axis=1, skipna=False))
</pre></td>
<td><img width="100%" src="./figures/missingdata/miss1-8data.png"></td>
<td><img width="60%" src="./figures/missingdata/miss1-8.png"></td>
<td>
ただし、skipna=Falseと明示的に指定すれば、NAがあれば演算結果もNAとなる。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-9</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
df = pd.DataFrame({'a':[1, -2, 3]})
print(df)
pd.options.mode.use_inf_as_na = False
print(df/0)
pd.options.mode.use_inf_as_na = True
print(df/0)
</pre></td>
<td><img width="40%" src="./figures/missingdata/miss1-9data.png"></td>
<td><img width="100%" src="./figures/missingdata/miss1-9.png"></td>
<td>
0割り算の結果をinfからNaNに変更することも可能。
</td>
</tr>
        
     
</tbody>
</table>

In [3]:
print('''
# 入力データ
# float型でNoneを指定すればNA(np.nan)になる。
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0 ]}
    )
>>> print(d)
''')
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0 ]}
)
#d = pd.DataFrame({'x':[1.0, None, 3.0, None], 'y':[1.5, None, None, 1.0], 'z':[pd.Period('2021-12-01', freq='D'), None ,pd.Period('2021-12-12', freq='D') ,pd.Period('2021-12-12', freq='D')]})
print(d)

print('''
# no.1-1
# isna()メソッドで、値がNAかどうかをTrue/Falseで出力できる。
>>> print(d.isna())''')
print(d.isna())

print('''
# no.1-2
# Seriesに対して利用することも可能
>>> print(d['x'].isna())''')
print(d['x'].isna())

print('''
# no.1-3
# notna()メソッドはisna()の逆の動きをする。
>>> print(d.notna())''')
print(d.notna())

print('''
# no.1-4
# []演算子と組み合わせればNAの行を選択することができる。
# x列がNAの行を選択する例。
>>> print(d[d['x'].isna()])''')
print(d[d['x'].isna()])

print('''
# no.1-5
# NAに対する演算はNA。
>>> print(d * 2)''')
print(d * 2)

print('''
# no.1-6
# NAに対する比較演算はFalse
>>> print(d > 0)''')
print(d > 0)

print('''
# no.1-7
# sum()などの集約メソッドについてはNAは無視される。
>>> print(d.sum(axis=1))''')
print(d.sum(axis=1))

print('''
# no.1-8
# ただし、skipna=Falseと明示的に指定すれば、NAがあれば演算結果もNAとなる。
>>> print(d.sum(axis=1, skipna=False))''')
print(d.sum(axis=1, skipna=False))

print('''
# no.1-9
# 0割り算の結果をinfからNaNに変更することも可能。
>>> df = pd.DataFrame({'a':[1, -2, 3]})
>>> pd.options.mode.use_inf_as_na = False
>>> print(df/0)
>>> pd.options.mode.use_inf_as_na = True
>>> print(df/0)''')
df = pd.DataFrame({'a':[1, -2, 3]})
print(df)
pd.options.mode.use_inf_as_na = False
print(df/0)
pd.options.mode.use_inf_as_na = True
print(df/0)



# 入力データ
# float型でNoneを指定すればNA(np.nan)になる。
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0 ]}
    )
>>> print(d)

     x    y    z
0  1.0  1.5  4.0
1  NaN  NaN  NaN
2  3.0  NaN  2.0
3  NaN  1.0  1.0

# no.1-1
# isna()メソッドで、値がNAかどうかをTrue/Falseで出力できる。
>>> print(d.isna())
       x      y      z
0  False  False  False
1   True   True   True
2  False   True  False
3   True  False  False

# no.1-2
# Seriesに対して利用することも可能
>>> print(d['x'].isna())
0    False
1     True
2    False
3     True
Name: x, dtype: bool

# no.1-3
# notna()メソッドはisna()の逆の動きをする。
>>> print(d.notna())
       x      y      z
0   True   True   True
1  False  False  False
2   True  False   True
3  False   True   True

# no.1-4
# []演算子と組み合わせればNAの行を選択することができる。
# x列がNAの行を選択する例。
>>> print(d[d['x'].isna()])
    x    y    z
1 NaN  NaN  NaN
3 NaN  1.0  1.0

# no.1-5
# NAに対する演算はNA。
>>> print(d * 2)
     x    y    z
0  2.0  3.0  8.0
1  NaN  NaN  NaN
2  6.0 

---
# 2. null値を含む行の削除
以下の例は、dropna(axis=1)のように、axisパラメータを付け加えることで列の削除にも適用できる。

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0 ]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th>入力</th><th>出力</th><th>備考</th></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.dropna())
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss2-1data.png"></td>
<td><img width="80%" src="./figures/missingdata/miss2-1.png"></td>
<td>
dropna()でNAを含む行を削除できる。DataFrameに適用すると、いずれかの列がNAであれば削除される。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.dropna(how='all'))
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss2-2data.png"></td>
<td><img width="80%" src="./figures/missingdata/miss2-2.png"></td>
<td>
how='all'とすれば、全ての列がNAである行のみが削除される。
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.dropna(subset=['x', 'y'], how='all'))
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss2-3data.png"></td>
<td><img width="80%" src="./figures/missingdata/miss2-3.png"></td>
<td>
列の一部のNAを評価したければsubset=を用いる。２つの列x,yがいずれもNAの行を削除する。
</td>
</tr>
 
</tbody>
</table>

In [4]:
print('''
# 入力データ
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0 ]}
    )
>>> print(d)''')
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0 ]}
)
print(d)

print('''
# no.2-1
# dropna()でNAを含む行を削除できる。
# DataFrameに適用すると、いずれかの列がNAであれば削除される。
>>> print(d.dropna())''')
print(d.dropna())

print('''
# no.2-2
# how='all'とすれば、全ての列がNAである行のみが削除される。
>>> print(d.dropna(how='all'))''')
print(d.dropna(how='all'))

print('''
# no.2-3
# 列の一部のNAを評価したければsubset=を用いる。
# ２つの列x,yがいずれもNAの行を削除する。
>>> print(d.dropna(subset=['x', 'y'], how='all'))''')
print(d.dropna(subset=['x', 'y'], how='all'))



# 入力データ
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0 ]}
    )
>>> print(d)
     x    y    z
0  1.0  1.5  4.0
1  NaN  NaN  NaN
2  3.0  NaN  2.0
3  NaN  1.0  1.0

# no.2-1
# dropna()でNAを含む行を削除できる。
# DataFrameに適用すると、いずれかの列がNAであれば削除される。
>>> print(d.dropna())
     x    y    z
0  1.0  1.5  4.0

# no.2-2
# how='all'とすれば、全ての列がNAである行のみが削除される。
>>> print(d.dropna(how='all'))
     x    y    z
0  1.0  1.5  4.0
2  3.0  NaN  2.0
3  NaN  1.0  1.0

# no.2-3
# 列の一部のNAを評価したければsubset=を用いる。
# ２つの列x,yがいずれもNAの行を削除する。
>>> print(d.dropna(subset=['x', 'y'], how='all'))
     x    y    z
0  1.0  1.5  4.0
2  3.0  NaN  2.0
3  NaN  1.0  1.0


---
# 3. null値の置換(fillna, combine_first)

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>
 
<tr><th align="left">no.</th><th align="left">実行コード</th><th>入力</th><th>出力</th><th>備考</th></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.fillna(0))
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss3-1data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss3-1.png"></td>
<td>
NAを0に置換する。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.fillna({'x':-1, 'y':0, 'z':999}))
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss3-2data.png"></td>
<td><img width="80%" src="./figures/missingdata/miss3-2.png"></td>
<td>
辞書で列別に置換する値を指定することも可能。
</td>
</tr>
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.fillna(method='ffill'))
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss3-3data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss3-3.png"></td>
<td>
method='ffill'とすることで、同じ列のNAでない直近の値で埋める。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(d.fillna(method='bfill'))
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss3-4data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss3-4.png"></td>
<td>
method='bfill'は、ffillと逆で後ろから見ての直近の値で埋める。
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-5</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.copy()
dd['x'] = dd['x'].combine_first(pd.Series([1, 2, 3, 4]))
print(dd)
</pre></td>
<td><img width="100%" src="./figures/missingdata/miss3-5data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss3-5.png"></td>
<td>
combine_firstを用いることで、同じサイズの他のSeriesで埋めることができる。<br>
x列に、同サイズのSeries[1,2,3,4]を重ねて、x列NAの部分を該当する位置の値で置換する。
</td>
</tr>
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-6</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.copy()
dd['x'] = dd['x'].combine_first(dd['z'])
print(dd)
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss3-6data.png"></td>
<td><img width="70%" src="./figures/missingdata/miss3-6.png"></td>
<td>
x列のNAをz列で置換する。
</td>
</tr>
      
</tbody>
</table>

In [5]:
print('''
# 入力データ
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0]}
    )
>>> print(d)''')
d = pd.DataFrame(
    {'x':[1.0, None, 3.0, None],
     'y':[1.5, None, None, 1.0],
     'z':[4.0, None, 2.0, 1.0]}
)
print(d)

print('''
# no.3-1
# NAを0に置換する。
>>> print(d.fillna(0))''')
print(d.fillna(0))

print('''
# no.3-2
# 辞書で列別に値を指定することも可能。
>>> print(d.fillna({'x':-1, 'y':0, 'z':999}))''')
print(d.fillna({'x':-1, 'y':0, 'z':999}))

print('''
# no.3-3
# method='ffill'とすることで、同じ列のNAでない直近の値で埋める。
>>> print(d.fillna(method='ffill'))''')
print(d.fillna(method='ffill'))

print('''
# no.3-4
# method='bfill'は、ffillと逆で後ろから見ての直近の値で埋める。
>>> print(d.fillna(method='bfill'))''')
print(d.fillna(method='bfill'))

print('''
# no.3-5
# combine_firstを用いることで、同じサイズの他のSeriesで埋めることができる。
# x列に、同サイズのSeries[1,2,3,4]を重ねて、x列NAの部分を該当する位置の値で置換する。
>>> dd = d.copy()
>>> dd['x'] = dd['x'].combine_first(pd.Series([1, 2, 3, 4]))
>>> print(dd)''')
dd = d.copy()
dd['x'] = dd['x'].combine_first(pd.Series([1, 2, 3, 4]))
print(dd)

print('''
# no.3-6
# x列のNAをz列で置換する。
>>> dd = d.copy()
>>> dd['x'] = dd['x'].combine_first(dd['z'])
>>> print(dd)''')
dd = d.copy()
dd['x'] = dd['x'].combine_first(dd['z'])
print(dd)



# 入力データ
>>> d = pd.DataFrame(
        {'x':[1.0, None, 3.0, None],
         'y':[1.5, None, None, 1.0],
         'z':[4.0, None, 2.0, 1.0]}
    )
>>> print(d)
     x    y    z
0  1.0  1.5  4.0
1  NaN  NaN  NaN
2  3.0  NaN  2.0
3  NaN  1.0  1.0

# no.3-1
# NAを0に置換する。
>>> print(d.fillna(0))
     x    y    z
0  1.0  1.5  4.0
1  0.0  0.0  0.0
2  3.0  0.0  2.0
3  0.0  1.0  1.0

# no.3-2
# 辞書で列別に値を指定することも可能。
>>> print(d.fillna({'x':-1, 'y':0, 'z':999}))
     x    y      z
0  1.0  1.5    4.0
1 -1.0  0.0  999.0
2  3.0  0.0    2.0
3 -1.0  1.0    1.0

# no.3-3
# method='ffill'とすることで、同じ列のNAでない直近の値で埋める。
>>> print(d.fillna(method='ffill'))
     x    y    z
0  1.0  1.5  4.0
1  1.0  1.5  4.0
2  3.0  1.5  2.0
3  3.0  1.0  1.0

# no.3-4
# method='bfill'は、ffillと逆で後ろから見ての直近の値で埋める。
>>> print(d.fillna(method='bfill'))
     x    y    z
0  1.0  1.5  4.0
1  3.0  1.0  2.0
2  3.0  1.0  2.0
3  NaN  1.0  1.0

# no.3-5
# combine_firstを用いることで、同じサイズの他のSeriesで埋めることができる。
# x列に、同サイズのSeries[1,2,3,4]を重ねて、x列NAの部分を該当する位置の

---
## 4. NAの数値補完(interpolate)

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
a = pd.Series([4, 8, 2, None, None, 3])
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th>備考</th></tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>4-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(a.interpolate())
</pre></td>
<td><img width="40%" src="./figures/missingdata/miss4-1data.png"></td>
<td><img width="50%" src="./figures/missingdata/miss4-1.png"></td>
<td>
NAを挟んで前後の数値からNAを線形補間する(method='linear')
</td>
</tr>
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>4-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.Series([4, 8, 2, None, 3]).interpolate(method='spline', order=2))
</pre></td>
<td><img width="30%" src="./figures/missingdata/miss4-2data.png"></td>
<td><img width="50%" src="./figures/missingdata/miss4-2.png"></td>
<td>
indexをx軸、Seriesの値をy軸とした時の2次のスプライン補間を実施する(indexをx軸とするのでmulti indexでは利用不可)。
methodに指定できるパラメータ: 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial'
これらのパラメータはscipyのinterpolate.interp1dメソッドに渡される。詳細は次のサイトのindパラメータを参照のこと。
<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html">scipy.interpolate.interp1d</a>
</td>
     
</tbody>
</table>

In [35]:
print('''
# 入力データ
>>> a = pd.Series([4, 8, 2, None, None, 3])
>>> print(a)''')
a = pd.Series([4, 8, 2, None, None, 3])
print(a)

print('''
# no.4-1
# NAを挟んで前後の数値からNAを線形補間する(method='linear')。
>>> print(a.interpolate())''')
print(a.interpolate())

print('''
# no.4-2
# indexをx軸、Seriesの値をy軸とした時の2次のスプライン補間(indexをx軸とするのでmulti indexでは利用不可)
# methodに指定できるパラメータ: 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial'
# これらのパラメータはscipyのinterpolate.interp1dメソッドに渡される。詳細は以下のサイトを参照のこと。
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html
>>> print(pd.Series([4, 8, 2, None, 3]).interpolate(method='spline', order=2))''')
print(pd.Series([4, 8, 2, None, 3]).interpolate(method='spline', order=2))



# 入力データ
>>> a = pd.Series([4, 8, 2, None, None, 3])
>>> print(a)
0    4.0
1    8.0
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

# no.4-1
# NAを挟んで前後の数値からNAを線形補間する(method='linear')。
>>> print(a.interpolate())
0    4.000000
1    8.000000
2    2.000000
3    2.333333
4    2.666667
5    3.000000
dtype: float64

# no.4-2
# indexをx軸、Seriesの値をy軸とした時の2次のスプライン補間(indexをx軸とするのでmulti indexでは利用不可)
# methodに指定できるパラメータ: 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial'
# これらのパラメータはscipyのinterpolate.interp1dメソッドに渡される。詳細は以下のサイトを参照のこと。
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html
>>> print(pd.Series([4, 8, 2, None, 3]).interpolate(method='spline', order=2))
0    4.000000
1    8.000000
2    2.000000
3    1.536056
4    3.000000
dtype: float64


---
## 5. 条件指定によるNAへの置換
### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame({'x':[1., 2., 6., 4.],'y':[10., 3., 8., 4]})
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th>備考</th></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>5-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.copy()
dd.loc[dd['x'] >= 6, 'x'] = None
print(dd)
</pre></td>
<td><img width="70%" src="./figures/missingdata/miss5-1data.png"></td>
<td><img width="50%" src="./figures/missingdata/miss5-1.png"></td>
<td>
NAをセットするにはNoneもしくはnp.nanを代入すればよい。x列が6以上のx列の値をNAを代入する例。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>5-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.where(d>=6)
print(dd)
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss5-2data.png"></td>
<td><img width="50%" src="./figures/missingdata/miss5-2.png"></td>
<td>
DataFrame全体に対して条件によるNA置換を行うのであればweher()メソッドが便利。DataFrame.where()は、与えた条件に一致しない値をNAに置換する。6以上はそのままに、6より小さい値をNAに置換する。
</td>
</tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>5-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.mask(d>=6)
print(dd)
</pre></td>
<td><img width="80%" src="./figures/missingdata/miss5-3data.png"></td>
<td><img width="50%" src="./figures/missingdata/miss5-3.png"></td>
<td>
逆にmask()は、与えた条件に一致する値をNAに置換する。6以上の値をNAに置換する。
</td>
</tr>
    
</tbody>
</table>

In [6]:
print('''
# 入力データ
>>> d = pd.DataFrame({'x':[1., 2., 6., 4.],'y':[10., 3., 8., 4]})
>>> print(d)''')
d = pd.DataFrame({'x':[1., 2., 6., 4.],'y':[10., 3., 8., 4]})
print(d)

print('''
# no.5-1
# NAをセットするにはNoneもしくはnp.nanを代入すればよい。
# x列が6以上のx列の値をNAを代入する例。
>>> dd = d.copy()
>>> dd.loc[dd['x'] >= 6, 'x'] = None
>>> print(dd)''')
dd = d.copy()
dd.loc[dd['x'] >= 6, 'x'] = None
print(dd)

print('''
# no.5-2
# DataFrame全体に対して条件によるNA置換を行うのであればweher()メソッドが便利。
# DataFrame.where()は、与えた条件に一致しない値をNAに置換する。
# 6以上はそのままに、６より小さい値をNAに置換する。
>>> dd = d.where(d>=6)
>>> print(dd)''')
dd = d.where(d>=6)
print(dd)

print('''
# no.5-3
# 逆にmask()は、与えた条件に一致する値をNAに置換する。
# 6以上の値をNAに置換する。
>>> dd = d.mask(d>=6)
>>> print(dd)''')
dd = d.mask(d>=6)
print(dd)


# 入力データ
>>> d = pd.DataFrame({'x':[1., 2., 6., 4.],'y':[10., 3., 8., 4]})
>>> print(d)
     x     y
0  1.0  10.0
1  2.0   3.0
2  6.0   8.0
3  4.0   4.0

# no.5-1
# NAをセットするにはNoneもしくはnp.nanを代入すればよい。
# x列が6以上のx列の値をNAを代入する例。
>>> dd = d.copy()
>>> dd.loc[dd['x'] >= 6, 'x'] = None
>>> print(dd)
     x     y
0  1.0  10.0
1  2.0   3.0
2  NaN   8.0
3  4.0   4.0

# no.5-2
# DataFrame全体に対して条件によるNA置換を行うのであればweher()メソッドが便利。
# DataFrame.where()は、与えた条件に一致しない値をNAに置換する。
# 6以上はそのままに、６より小さい値をNAに置換する。
>>> dd = d.where(d>=6)
>>> print(dd)
     x     y
0  NaN  10.0
1  NaN   NaN
2  6.0   8.0
3  NaN   NaN

# no.5-3
# 逆にmask()は、与えた条件に一致する値をNAに置換する。
# 6以上の値をNAに置換する。
>>> dd = d.mask(d>=6)
>>> print(dd)
     x    y
0  1.0  NaN
1  2.0  3.0
2  NaN  NaN
3  4.0  4.0
